# CONTINUOUS TRAINING

When a face gets a similarity thats higher than 75%, the model is trained to increase the accuracy to that face.

In [166]:
import warnings
warnings.filterwarnings("ignore")

import cv2
import numpy as np
from PIL import Image

## DB

In [167]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from psycopg2.errors import UniqueViolation

import json
import dotenv
import os
dotenv.load_dotenv()
DB_URL = os.getenv("DB_URL")

engine = create_engine(DB_URL)
Base = declarative_base()
Session = sessionmaker(bind=engine)

class Faces(Base):
    __tablename__ = 'faces'
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, unique=True, nullable=False)
    embeddings = Column(String, unique=False, nullable=False)
    
Base.metadata.create_all(bind=engine)

READ ALL FACES FROM DB

In [168]:
def read_all_embeddings():
    with Session() as session:
        try:
            user_list = {}
            users = session.query(Faces).all()
            for user in users:
                try:
                    name = user.name
                    embeddings = np.array(json.loads(user.embeddings), dtype=np.float32)
                    user_list[name] = embeddings
                except:
                    continue
            return user_list
        except Exception as e:
            print(f'Error: {e}')
            return None

CREATE EMBEDDINGS

In [169]:
def create_embeddings(name, embeddings):
    try:
        with Session() as session:
            embeddings = json.dumps(embeddings.tolist())
            user = Faces(name=name, embeddings=embeddings)
            session.add(user)
            session.commit()

    except Exception as e:
        if UniqueViolation:
            print(f"\nFace with name {user.name} already exists\n")

READ EMBEDDINGS

In [170]:
def read_embeddings(name):
    try:
        with Session() as session:
            user = session.query(Faces).filter(Faces.name == name).first()
            if user:
                embeddings = np.array(json.loads(user.embeddings), dtype=np.float32)
                return embeddings
            else:
                return f'No face with name {name} found'

    except Exception as e:
        print(f"\nError getting embedding: {e}\n")

UPDATE EMBEDDINGS

In [171]:
def update_embeddings(name, embeddings):
    try:
        with Session() as session:
            user = session.query(Faces).filter(Faces.name == name).first()
            if user:
                user.embeddings = json.dumps(embeddings.tolist())
                session.commit()
            else:
                return Exception("User not found")

    except Exception as e:
        print(f"\nError getting embedding: {e}\n")

DELETE EMBEDDINGS

In [172]:
def delete_embeddings(name):
    try:
        with Session() as session:
            user = session.query(Faces).filter(Faces.name == name).first()
            if user:
                session.delete(user)
                session.commit()
            else:
                return Exception("User not found")
    except Exception as e:
        print(f"\nError getting embedding: {e}\n")

## FUNCTIONS

In [173]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import random

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

set_seed()

COMPARE EMBEDDINGS

In [174]:
from sklearn.metrics.pairwise import cosine_similarity

def compare_embeddings(features1, features2):
    """
        Compare the similarity between two features.
        The similarity is calculated using cosine similarity.
        Use extract_features to get the features before comparing.
    """
    if features1 is None or features2 is None:
        return None
    similarity = cosine_similarity(features1, features2)[0][0]
    
    return similarity

DETECT FACE

In [175]:
import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh

def detect_face(frame):
    face_mesh = mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w = frame.shape[:2]
            x_min = w
            y_min = h
            x_max = y_max = 0
            for landmark in face_landmarks.landmark:
                x, y = int(landmark.x * w), int(landmark.y * h)
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x)
                y_max = max(y_max, y)
            return (x_min, y_min, x_max, y_max)

PTH PROCESSING

In [176]:
from torchvision import transforms

def pth_processing(fp):
    class PreprocessInput(nn.Module):
        def forward(self, x):
            x = x.to(torch.float32)
            x = torch.flip(x, dims=(0,))
            x[0, :, :] -= 91.4953
            x[1, :, :] -= 103.8827
            x[2, :, :] -= 131.0912
            return x

    def get_img_torch(img):
        ttransform = transforms.Compose([
            transforms.PILToTensor(),
            PreprocessInput()
        ])
        img = img.resize((224, 224), Image.Resampling.NEAREST)
        img = ttransform(img)
        img = torch.unsqueeze(img, 0).to(device)
        return img
    return get_img_torch(fp)

FACE TRANSFORMER

In [177]:
class FaceTransformer(nn.Module):
    def __init__(self, backbone, d_model=512, nhead=8, num_layers=3):
        super(FaceTransformer, self).__init__()
        self.backbone = backbone
        self.d_model = d_model
        
        self.projection = nn.Linear(512, d_model)
        
        self.pos_encoder = nn.Parameter(torch.randn(1, 49, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model * 49, 512)  # Final embedding size

    def forward(self, x):
        features = self.backbone.extract_features(x)
        features = torch.nn.functional.relu(features)
        
        features = features.view(features.size(0), 512, -1)
        features = self.projection(features.permute(0, 2, 1))
        
        features = features + self.pos_encoder
        
        output = self.transformer_encoder(features.permute(1, 0, 2))
        output = output.permute(1, 2, 0).flatten(1)
        return self.fc(output)

EXTRACT FEATURES

In [178]:
pth_backbone_model = torch.jit.load('model/torchscript_model_0_66_49_wo_gl.pth').to(device)
model = FaceTransformer(pth_backbone_model).to(device)

def extract_features(image):
    """
        Extract features from a cv2 image.
        The code itself detects the face in the image and extracts the features.
    """
    face = detect_face(image)
    if face is None:
        return None
    x1, y1, x2, y2 = face
    face_img = Image.fromarray(cv2.cvtColor(image[y1:y2, x1:x2], cv2.COLOR_BGR2RGB))
    face_tensor = pth_processing(face_img)
    with torch.no_grad():
        features = model(face_tensor)
        
    return features.cpu().numpy()

## MAIN

In [176]:
image = cv2.imread('src/photo2.jpeg')
features = extract_features(image)

faces = read_all_embeddings()

for name, embeddings in faces.items():
    best_similarity = 0
    best_name = ''
    similarity = compare_embeddings(features, embeddings)
    if similarity > best_similarity:
        best_similarity = similarity
        best_name = name
        
print(f'Predicted: {best_name} with similarity {best_similarity}')

0.9185674


I0000 00:00:1723899240.840904 15709080 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1723899240.844080 15730369 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723899240.847521 15730369 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1723899240.962889 15709080 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1723899240.963866 15730391 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723899240.965549 15730391 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
